In [11]:
#TODO: None of this has been tested as I didn't get the chance to export data from the processing_test file.
import subprocess
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import folium

In [ ]:
def get_git_root():
    try:
        root_dir = (
            subprocess.check_output(["git", "rev-parse", "--show-toplevel"])
            .strip()
            .decode("utf-8")
        )
        return root_dir
    except subprocess.CalledProcessError:
        raise Exception("This is not a Git repository")


# Get the Git repository's root directory
repo_root = get_git_root()
subfolder='data'
###FIXME: Maybe better folders for the data, the file_paths would need to be changed in that case
file_path = os.path.join(repo_root, subfolder, "SNGE8RFL.shp")
gps_df=gpd.read_file(file_path)

In [ ]:
###Calculate the wealth index average for each cluster with weights based on the household sample weight
household_wealth_index_df=pd.read_csv(os.path.join(repo_root, "Wealth_Index_DF.csv"))
def weighted_average(values, weights):
    return np.average(values, weights=weights) 

cluster_wealth=household_wealth_index_df.groupby('hv001').apply(
    lambda x:pd.series({'wealth_index':weighted_average(x['wealth_index'],x['hv005'])})
).reset_index()

In [ ]:
gps_df=pd.merge(gps_df, cluster_wealth,on='hv001',how='left') #Adding the wealth index to the gps data


In [12]:
senegal_filepath=os.path.join(repo_root, subfolder,'Senegal_Shapefile', "gadm41_SEN_0.shp")
senegal_boundary=gpd.read_file(senegal_filepath)

In [ ]:
###Creating the geometry and the dataframe that includes the geometry
geometry=[Point(xy) for xy in zip(gps_df['LONGNUM'],gps_df['LATNUM'])]
gdf=gpd.GeoDataFrame(gps_df,geometry=geometry)

###adding a buffer based on ubran or rural
def create_errorbounds(row):
    if row["URBAN_RURA"]=='U':
        return row['geometry'].buffer(2000)
    elif row["URBAN_RURA"]=='R':
        return row['geometry'].buffer(10000)
    else:
        print(f"Error, can't determine urban or rural for row {row}. Exiting")
        exit()

gdf['buffer']=gdf.apply(create_errorbounds,axis=1)

###Clipping the buffer to the Senegal boundary
gdf['buffer_clipped']=gdf['buffer'].apply(lambda x: x.intersection(senegal_boundary.geometry.unary_union)) 

buffer_gdf=gpd.GeoDataFrame(gdf,geometry=gdf['buffer_clipped'])

###Checking for overlapping regions
gdf_within_buffers=gpd.sjoin(gdf,buffer_gdf,how='left',op='within')

###For overlapping regions, averaging the wealth index
gdf_within_buffers['wealth_index']=gdf_within_buffers.groupby('index_right')['wealth_index'].transform('mean')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Plot Senegal boundary
senegal_boundary.plot(ax=ax, color="lightgray", edgecolor="black")

# Plot the points with averaged wealth index values
gdf_within_buffers.plot(ax=ax, color="blue", alpha=0.5, markersize=5)

gdf_within_buffers.plot(
    column="wealth_index",
    ax=ax,
    legend=True,
    legend_kwds={"label": "Average Wealth Index", "orientation": "horizontal"},
)

plt.title("Averaged Wealth Index Across Senegal")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
###Trying an interactive map centered on Senegal
m = folium.Map(location=[14.5, -14], zoom_start=7)  # Center on Senegal
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row["LATNUM"], row["LONGNUM"]],
        radius=5,
        color="blue" if row["URBAN_RURA"] == "U" else "green",
        fill=True,
        fill_color="blue" if row["URBAN_RURA"] == "U" else "green",
        fill_opacity=0.6,
        popup=f"Region: {row['hv001']} | Wealth Index: {row['wealth_index']}",
    ).add_to(m)

m.save("wealth_index_map.html")